In [1]:
library(grid)
library(rpart)
library(rpart.plot)
library(partykit)
library(lattice)
library(ggplot2)
library(caret)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
set.seed(1)

In [3]:
jRoadType <- c("都市間高速", "都市高速", "有料道路", "国道", "県道", "主要地方道", "一般道1、一般道2、一般道3", "その他")
jHighSpeeds <- c("都市間高速", "都市高速", "有料道路")

In [4]:
printf <- function(...) cat(sprintf(...))

In [5]:
# 利用する特徴量を列挙
valuables <- c("RoadType","CurveAverage","Speed", "Curve100", "Curve150","MaxSpeed","RiskFactor","Curve","DistSignal","Pitch","AheadDistance","AverageVelocity","TimeToCollision","AccelerationSpeed", "Engine", "SteeringAngle", "TimeHeadway", "Jerk", "LaneCount", "DiffAvgSpeed", "EmptinessOfRoad", "RoadFactor")

In [6]:
# CV value を計算
# dfx: 使用するデータ
# verbose: ログを出力するかどうか
CV <- function(dfx, verbose=FALSE) {        
    # k=5 束の fold を作成
    folds <- createFolds(dfx$flag, k=5)
    
    # 各 fold における結果を格納する変数
    count <- 1
    errs <- c()       # CV 値
    reds <- c()     # 長時間平均 (Red だと予測して実際に Red だったもの)
    blues <- c()    # 長時間平均 (Blue だと予測して実際に Blue だったもの)
    for (ids in folds) {
        train <- dfx[-ids, ]
        test <- dfx[ids, ]
        fit <- rpart(flag ~ ., data=train, method="class", cp=0.022) # Decision Tree で分類
        p <- predict(fit, newdata=test)                                              # テストデータで予測
        predictedFlags <- colnames(p)[max.col(p, ties.method = "first")]   # 予測確率が一番高いものを選択
        
        # 予測した結果を用いて長期平均を計算
        if (verbose) {
            printf("Fold%d\n", count)
            result <- correctVsPredict(test, predictedFlags, verbose)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
            count <- count + 1
            printf("\n")
        } else {
            result <- correctVsPredict(test, predictedFlags, verbose)
            reds <- c(reds, result[1])
            blues <- c(blues, result[2])
        }
        
        nerr <- sum(predictedFlags != test$flag)
        errs <- c(errs, nerr / nrow(test))
    }
    
    c(mean(errs), mean(reds), mean(blues))
}

In [7]:
# 長時間平均を計算する
correctVsPredict <- function(test, predictedFlags, verbose=FALSE) {    
    # for Red
    predictedRedRows <- test[predictedFlags == "Red", ]
    nCorrectReds <- sum(predictedRedRows$flag == 'Red')
    nPredictedReds <- nrow(predictedRedRows)
    
    # for Blue
    predictedBlueRows <- test[predictedFlags == "Blue", ]
    nCorrectBlues <- sum(predictedBlueRows$flag == 'Blue')
    nPredictedBlues <- nrow(predictedBlueRows)
        
    if (verbose) {
        printf("As for Red: correct/predict = %d/%d = %f\n", nCorrectReds, nPredictedReds, nCorrectReds / nPredictedReds)
        printf("As for Blue: correct/predict = %d/%d = %f\n", nCorrectBlues, nPredictedBlues, nCorrectBlues / nPredictedBlues)  
    }    
    
    c(nCorrectReds/nPredictedReds, nCorrectBlues/nPredictedBlues)
}

In [8]:
# 全体のうち Red と Blue がどれぐらいの割合かを出力する
printRedRatios <- function(dfx) {
    nRed <- nrow(dfx[dfx$flag == "Red", ])
    nAll <- nrow(dfx)
    printf("Red/All = %d/%d = %f\n", nRed, nAll, nRed/nAll)
    printf("1 - Red/All = %d/%d = %f\n", nAll - nRed, nAll, 1 - nRed/nAll)
}

# Predict Reds

In [9]:
df3 <- read.csv("../data/middle/sp5.csv", stringsAsFactors=FALSE)

In [10]:
df3$flag[df3$flag == "RedA"] <- "Red"
df3$flag[df3$flag == "RedB"] <- "Red"
df3$flag[df3$flag == "BlueA"] <- "Blue"
df3$flag[df3$flag == "BlueB"] <- "Blue"
df3$flag <- as.factor(df3$flag)

In [11]:
allFeatures <- c(colnames(df3))

In [12]:
# 存在する RoadType だけ取り出す
roadTypes <- unique(df3$RoadType)

In [13]:
verbose = FALSE

In [26]:
# 使用する特徴量だけ抜き取る
df3 <- df3[, c(valuables, "flag")]

In [14]:
# 各道路タイプに対して CV 値, 長時間平均を計算
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    result <- CV(dfx, verbose=verbose)
    printRedRatios(dfx)
    printf("Red: Mean correct/predict = %f\n", result[2])
    printf("Blue: Mean correct/predict = %f\n", result[3])
    printf("CV value: %f", result[1])
    printf("\n\n")
}

RoadType: 7 (その他)
Red/All = 25/27 = 0.925926
1 - Red/All = 2/27 = 0.074074
Red: Mean correct/predict = 0.933333
Blue: Mean correct/predict = NaN
CV value: 0.066667

RoadType: 6 (一般道1、一般道2、一般道3)
Red/All = 90/189 = 0.476190
1 - Red/All = 99/189 = 0.523810
Red: Mean correct/predict = 0.588182
Blue: Mean correct/predict = 0.608465
CV value: 0.407255

RoadType: 4 (県道)
Red/All = 36/71 = 0.507042
1 - Red/All = 35/71 = 0.492958
Red: Mean correct/predict = 0.575794
Blue: Mean correct/predict = 0.547857
CV value: 0.437143

RoadType: 5 (主要地方道)
Red/All = 154/303 = 0.508251
1 - Red/All = 149/303 = 0.491749
Red: Mean correct/predict = 0.595961
Blue: Mean correct/predict = 0.585389
CV value: 0.409126

RoadType: 3 (国道)
Red/All = 202/324 = 0.623457
1 - Red/All = 122/324 = 0.376543
Red: Mean correct/predict = 0.680126
Blue: Mean correct/predict = 0.495163
CV value: 0.376250

RoadType: 0 (都市間高速)
Red/All = 55/81 = 0.679012
1 - Red/All = 26/81 = 0.320988
Red: Mean correct/predict = 0.695455
Blue: Mean corr

# Plot Tree

In [15]:
# 木の表示
set.seed(1)

In [16]:
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    
    fit <- rpart(flag ~., data=dfx, method="class", cp=0.022)

    # i = 7 のときは木が作れずエラーになるので回避
    if (i != 7) {
        fname = paste("Dtree-RoadType-", i, ".png", sep="")
        png(fname, height=960, width=960, res=144)   # res=144 で高解像度で保存
        plot(fit, uniform=TRUE, main="Classification Tree")
        text(fit, use.n=TRUE, all=TRUE, cex=.6)
        dev.off()
    }
}

RoadType: 7 (その他)
RoadType: 6 (一般道1、一般道2、一般道3)
RoadType: 4 (県道)
RoadType: 5 (主要地方道)
RoadType: 3 (国道)
RoadType: 0 (都市間高速)
RoadType: 2 (有料道路)


# Variable importance

In [17]:
# 特徴量の重要度
set.seed(1)

In [18]:
for (i in roadTypes) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    
    fit <- rpart(flag ~ ., data=dfx, method="class", cp=0.022)
    
    # i = 7 のときは木が作れずエラーになるので回避
    if (i != 7) {
        summary(fit)
    }
}

RoadType: 7 (その他)
RoadType: 6 (一般道1、一般道2、一般道3)
Call:
rpart(formula = flag ~ ., data = dfx, method = "class", cp = 0.022)
  n= 189 

          CP nsplit rel error    xerror       xstd
1 0.27777778      0 1.0000000 1.0000000 0.07628962
2 0.07777778      1 0.7222222 0.8111111 0.07437329
3 0.06666667      2 0.6444444 0.8111111 0.07437329
4 0.05555556      3 0.5777778 0.7666667 0.07354307
5 0.02777778      4 0.5222222 0.7111111 0.07228889
6 0.02222222      6 0.4666667 0.7222222 0.07255947
7 0.02200000      7 0.4444444 0.7666667 0.07354307

Variable importance
            Pitch      DiffAvgSpeed        DistSignal             Speed 
               20                15                12                11 
    AheadDistance     SteeringAngle      CurveAverage AccelerationSpeed 
                7                 7                 6                 4 
         MaxSpeed          Curve100   AverageVelocity          Curve150 
                4                 3                 2                 2 
 

# Threshold = 10, 30

In [19]:
# さらに Threshold = 10, 30 で分割する
set.seed(1)

In [20]:
verbose = FALSE

In [21]:
velocities <- c("slow", "middle", "fast")
for (i in c(5, 3)) {
    threshold1 <- 10
    threshold2 <- 30
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    # dfx を 3 つに分割
    dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
    dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
    dfx3 <- dfx %>% filter(AverageVelocity > threshold2)

    for (j in c(1, 2, 3)) {
        # CV 値, 長時間平均, 短時間平均を計算
        if (j == 1) {
            result <- CV(dfx1, verbose=verbose)        
        } else if (j == 2) {
            result <- CV(dfx2, verbose=verbose)        
        } else {
            result <- CV(dfx3, verbose=verbose)        
        }

        printf("Velocity: %s\n", velocities[j])
        printRedRatios(dfx1)        
        printf("Red: Mean correct/predict = %f\n", result[2])
        printf("Blue: Mean correct/predict = %f\n", result[3])
        printf("CV value: %f", result[1])        
        printf("\n\n")
    }
}

RoadType: 5 (主要地方道)
Velocity: slow
Red/All = 33/69 = 0.478261
1 - Red/All = 36/69 = 0.521739
Red: Mean correct/predict = 0.399242
Blue: Mean correct/predict = 0.476623
CV value: 0.549670

Velocity: middle
Red/All = 33/69 = 0.478261
1 - Red/All = 36/69 = 0.521739
Red: Mean correct/predict = 0.631472
Blue: Mean correct/predict = 0.620301
CV value: 0.376984

Velocity: fast
Red/All = 33/69 = 0.478261
1 - Red/All = 36/69 = 0.521739
Red: Mean correct/predict = 0.561270
Blue: Mean correct/predict = 0.366667
CV value: 0.505152

RoadType: 3 (国道)
Velocity: slow
Red/All = 34/52 = 0.653846
1 - Red/All = 18/52 = 0.346154
Red: Mean correct/predict = 0.758889
Blue: Mean correct/predict = NaN
CV value: 0.349091

Velocity: middle
Red/All = 34/52 = 0.653846
1 - Red/All = 18/52 = 0.346154
Red: Mean correct/predict = 0.613833
Blue: Mean correct/predict = 0.274417
CV value: 0.484097

Velocity: fast
Red/All = 34/52 = 0.653846
1 - Red/All = 18/52 = 0.346154
Red: Mean correct/predict = 0.569359
Blue: Mean cor

# Plot Tree

In [22]:
# 木を表示
set.seed(1)

In [23]:
threshold1 <- 10
threshold2 <- 30

In [24]:
for (i in c(5, 3)) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
    dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
    dfx3 <- dfx %>% filter(AverageVelocity > threshold2)
    
    for (j in c(1, 2, 3)) {
        if (j == 1) {
            fit <- rpart(flag ~ ., data=dfx1, method="class", cp=0.022)
        } else if (j == 2) {
            fit <- rpart(flag ~ ., data=dfx2, method="class", cp=0.022)
        } else {
            fit <- rpart(flag ~ ., data=dfx3, method="class", cp=0.022)
        }
        
        fname = paste("Dtree-RoadType-", velocities[j], "-Velocity-", j, ".png", sep="")
        png(fname, height=960, width=960, res=144)
        plot(fit, uniform=TRUE, main="Classification Tree")
        text(fit, use.n=TRUE, all=TRUE, cex=.6)
        dev.off()
    }
}

RoadType: 5 (主要地方道)
RoadType: 3 (国道)


# Importance

In [25]:
# さらに平均速度で分割したときの Tree の Variable Importance を表示
for (i in c(5, 3)) {
    printf("RoadType: %d (%s)\n", i, jRoadType[i+1])
    dfx <- df3[df3$RoadType == i, ]
    dfx1 <- dfx %>% filter(AverageVelocity <= threshold1)
    dfx2 <- dfx %>% filter(AverageVelocity > threshold1, AverageVelocity <= threshold2)
    dfx3 <- dfx %>% filter(AverageVelocity > threshold2)
    
    for (j in c(1, 2, 3)) {
        if (j == 1) {
            fit <- rpart(flag ~ ., data=dfx1, method="class", cp=0.022)
        } else if (j == 2) {
            fit <- rpart(flag ~ ., data=dfx2, method="class", cp=0.022)
        } else {
            fit <- rpart(flag ~ ., data=dfx3, method="class", cp=0.022)
        }
        summary(fit)
    }
}


RoadType: 5 (主要地方道)
Call:
rpart(formula = flag ~ ., data = dfx1, method = "class", cp = 0.022)
  n= 69 

          CP nsplit rel error   xerror      xstd
1 0.24242424      0 1.0000000 1.121212 0.1255271
2 0.12121212      1 0.7575758 1.090909 0.1257389
3 0.03030303      3 0.5151515 1.060606 0.1258447
4 0.02200000      4 0.4848485 0.969697 0.1255271

Variable importance
       DistSignal       TimeHeadway      CurveAverage          Curve100 
               16                14                10                 7 
            Curve   EmptinessOfRoad             Pitch             Speed 
                7                 6                 6                 5 
    AheadDistance          MaxSpeed     SteeringAngle      DiffAvgSpeed 
                5                 4                 4                 4 
        LaneCount            Engine   AverageVelocity AccelerationSpeed 
                3                 3                 3                 2 

Node number 1: 69 observations,    complexit